In [ ]:
import pandas as pd
import numpy as np
import os

## En datos/properati deben estar todos los archivos de ventas provistos por properati

In [ ]:
filesProperati = filter( lambda f: not f.startswith('.'), os.listdir("datos/properati"))

In [ ]:
dataFrames = []
asociados = dict()
#Este diccionario es para luego eliminar segun fechas erroneas
for file_name in filesProperati:
    
    newDataFrame = pd.read_csv("datos/properati/"+file_name)
    asociados[file_name] = newDataFrame
    dataFrames.append(newDataFrame)
       

In [ ]:
nulosStateName = 0
nulosPlaceWParentNames = 0
noCuentaConStateName = 0
noCuentaConPlaceWParentNames = 0
for df in dataFrames:
    try:
        nulosStateName += df.state_name.isnull().sum()
    except:
        noCuentaConStateName += 1
    try:
        nulosPlaceWParentNames += df.place_with_parent_names.isnull().sum()
    except:
        noCuentaConPlaceWParentNames += 1
print "Cantidad de registros con valor nulo en state_name: "+str(nulosStateName)
print "Cantidad de registros con valor nulo en place_w_parent_names: "+str(nulosPlaceWParentNames)
print "Cantidad de archivos que no cuentan con la columna state_name: "+str(noCuentaConStateName)
print "Cantidad de archivos que no cuentan con la columna place_w_parent_names: "+str(noCuentaConPlaceWParentNames)

## Dado los resultados, para quedarnos solo con las propiedades de Capital Federal y GBA tendremos que utilizar la columna 'place_with_parent_names'

In [ ]:
frecuencias = {}
for df in dataFrames:
    parentNames = df['place_with_parent_names']
    for padres in parentNames:
        zona = padres.split("|")[2]
        if zona not in frecuencias:
            frecuencias[zona] = 1
        else:
            frecuencias[zona]+=1

In [ ]:
for zona in frecuencias: 
    print zona,frecuencias[zona]

## Interesan: Bs.As. G.B.A. Zona Norte, Bs.As. G.B.A. Zona Sur, Bs.As. G.B.A. Zona Oeste, Capital Federal


In [ ]:
def filtrarCapitalYGBA(serie):
    deseados = ["Bs.As. G.B.A. Zona Norte", "Bs.As. G.B.A. Zona Sur", "Bs.As. G.B.A. Zona Oeste", "Capital Federal"]
    booleans = []
    for item in serie:
        if item.split("|")[2] in deseados:
            booleans.append(True)
        else:
            booleans.append(False)
    return booleans

In [ ]:
filtrados =[]
for df in dataFrames:
    
    df = df[filtrarCapitalYGBA(df['place_with_parent_names'])]
    filtrados.append(df)

### Por las dudas chequeo que ahora las frecuencias sean las mismas y solo queden las zonas que interesan

In [ ]:
frecuencias = {}
for df in filtrados:
    parentNames = df['place_with_parent_names']
    for padres in parentNames:
        zona = padres.split("|")[2]
        if zona not in frecuencias:
            frecuencias[zona] = 1
        else:
            frecuencias[zona]+=1

In [ ]:
for key in frecuencias:
    print key, frecuencias[key]

# Ahora recupero datos, elimino algunas columnas, y demas.
# Primero miro como estan los datos, para ver si conviene trabajar con los precios en pesos argentinos o dolares. 

In [ ]:
def a(df, column_name):
    try:
        return(df[column_name].isnull().sum(),0)
    except:
        return (0,1)

In [ ]:
nulsPriceAproxLocalCurr = 0
nulsPriceAproxUsd = 0
nulsPricePerM2 = 0
nulsPricePerM2usd = 0

priceAproxLocalCurrInex = 0
priceAproxUsdInex = 0
pricePerM2Inex = 0
pricePerM2usdInex = 0

for df in filtrados:
    nuls, inex = a(df, "price_aprox_local_currency")
    nulsPriceAproxLocalCurr += nuls
    priceAproxLocalCurrInex += inex
    nuls,inex = a(df, "price_aprox_usd")
    nulsPriceAproxUsd += nuls
    priceAproxUsdInex += inex
    nuls, inex = a(df, "price_per_m2")
    nulsPricePerM2 += nuls
    pricePerM2Inex += inex
    nuls,inex = a(df, "price_usd_per_m2")
    nulsPricePerM2usd += nuls
    pricePerM2usdInex += inex

In [ ]:
print str(nulsPriceAproxLocalCurr)+" Nulls priceAproxLocalCurr"
print str(nulsPriceAproxUsd)+" Nulls priceAproxUsd"
print str(nulsPricePerM2)+" Nulls Price Per M2"
print str(nulsPricePerM2usd)+" Nulls pricePerM2USD"
print str(priceAproxLocalCurrInex)+" Inexistentes priceAproxLocalCurr"
print str(priceAproxUsdInex)+" Inexistentes priceAproxUsd"
print str(pricePerM2Inex)+" Inexistentes pricePerM2"
print str(pricePerM2usdInex)+" Inexistentes pricePerM2usd"

# Tambien miro los datos acerca de las superficies

In [ ]:
for df in filtrados:
    print df.columns
    print ""

## Observo que algunos archivos solo tienen surface_in_m2, no distingue entre si es cubierta o si es total, suponemos que es la superficie total. 

In [ ]:
nulsSuperficieTotal = 0
for df in filtrados:
    try:
        nulsSuperficieTotal += df.surface_in_m2.isnull().sum()
    except:
        nulsSuperficieTotal += df.surface_total_in_m2.isnull().sum()
nulsSuperficieTotal

# Recuperamos lo que podemos de cada registro

In [ ]:
#Primero le pongo el mismo nombre a la superficie total
for df in filtrados:
    try:
        df['surface_total_in_m2']
    except:
        df.rename(columns={'surface_in_m2':'surface_total_in_m2'},inplace=True)
    

# Recupero la informacion que pueda, precios en dolares y superficie

In [ ]:
def recuperar_superficie(rowProperati):
    if pd.isnull(rowProperati['surface_total_in_m2']):

        if (not((pd.isnull(rowProperati['price_aprox_usd']))) and (not(pd.isnull(rowProperati['price_usd_per_m2'])))):
            
            return rowProperati['price_aprox_usd']/rowProperati['price_usd_per_m2']
            #rowProperati['surface_total_in_m2'] =
        else:
            return rowProperati['surface_total_in_m2']
    else:
        return rowProperati['surface_total_in_m2']

def recuperar_precio_usd(rowProperati):
    
    if pd.isnull(rowProperati['price_aprox_usd']):
    
        if (not(pd.isnull(rowProperati['surface_total_in_m2'])) and (not(pd.isnull(rowProperati['price_usd_per_m2'])))):
            
            return rowProperati['surface_total_in_m2']*rowProperati['price_usd_per_m2']
            #rowProperati['price_aprox_usd'] = 
        else:
            return rowProperati['price_aprox_usd']
    else:
        return rowProperati['price_aprox_usd']

def recuperar_ppm2(rowProperati):
    if pd.isnull(rowProperati['price_usd_per_m2']):
        
        if (not(pd.isnull(rowProperati['price_aprox_usd'])) and (not(pd.isnull(rowProperati['surface_total_in_m2'])))):
        
            try:
                return rowProperati['price_aprox_usd']/rowProperati['surface_total_in_m2']
                #rowProperati['price_usd_per_m2'] = 
            except ZeroDivisionError:
                #Despues se filtra
                return 0
        else:
            return rowProperati['price_usd_per_m2']
    else:
        return rowProperati['price_usd_per_m2']

                

In [ ]:
for df in filtrados:
     df['surface_total_in_m2'] = df.apply(lambda row: recuperar_superficie(row),axis=1)
     df['price_aprox_usd'] = df.apply(lambda row: recuperar_precio_usd(row),axis=1)
     df['price_usd_per_m2'] = df.apply(lambda row: recuperar_ppm2(row),axis=1)

# Chequeo que se hayan efectuado cambios

In [ ]:
a = 0
b = 0
c = 0
for df in filtrados:
    a += df.price_aprox_usd.isnull().sum()
    b += df.price_usd_per_m2.isnull().sum()
    c += df.surface_total_in_m2.isnull().sum()

print str(a)+" nuls priceAproxUsd"
print str(c)+" nuls surface_total_in_m2"
print str(b)+" nuls pricePerM2usd"

## Se recuperaron datos acerca del precio aproximado en usd y el precio por m2 en usd. No se pudieron recuperar nuevos datos acerca de la superficie total

## Elimino regisros de cada archivo que sean de fechas que no corresponden

In [ ]:
for key in asociados:
    #properati-AR-2016-04-01-properties-sell.csv
        
    fecha = key.split('-')[2:5]
    fechaInicio = ('-').join(fecha)
    fechaFin = ('-').join(key.split('-')[2:4])
    fechaFin = fechaFin + '31'
    df = asociados[key]
    df = df [df.created_on >= fechaInicio & df.created_on <= fechaFin]

In [ ]:
unificacion = pd.concat(filtrados, axis=0, ignore_index=True)

In [ ]:
unificacion.describe()

In [ ]:
unificacion.drop(["Unnamed: 0", "extra", "properati_url", "image_thumbnail","country_name", "lat","lon","operation"], axis = 1, inplace= True)
unificacion.to_csv("properati - GBA Y CABA 2013-2017")